<a href="https://colab.research.google.com/github/DaisyLaw/Machine-Learning-practices/blob/main/Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet tf_keras

In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
import tf_keras
import tensorflow as tf
import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)

# Check for GPU
print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

In [ ]:
# Random seeds
def set_seed(seed=0):
    np.random.seed(seed)
    tf.random.set_seed(seed)
set_seed()

In [ ]:
train_df = pd.read_csv("/content/sample_data/Digit_Recognizer/Data/train.csv")
train_df.head()

In [ ]:
len(train_df)

In [ ]:
train_df['label'].unique()

In [ ]:
plt.hist(train_df['label'], bins=range(11), rwidth=0.6, align='left')
plt.xticks(range(10))
plt.title("Distribution of labels across training data")
plt.ylabel('Occurences')
plt.xlabel('Labels')
plt.show()

In [ ]:
X = train_df.iloc[:,1:]
X = X.values.reshape(-1, 28, 28, 1)
X = X/255.0
X.shape

In [ ]:
plt.figure(figsize=(6,6))
for num in range(0,25):
    plt.subplot(5,5,num+1)
    plt.imshow(X[num, :, :, :], interpolation = "none", cmap = plt.cm.binary)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
y = train_df["label"]
len(y)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=20,  # randomly rotate images in the range 20 degrees
        zoom_range = 0.2, # Randomly zoom image 20%
        width_shift_range=0.2,  # randomly shift images horizontally 20%
        height_shift_range=0.2,  # randomly shift images vertically 20%
        shear_range=0.2) #shear intensity 20%


datagen.fit(X)

In [ ]:
path_test = "/content/sample_data/Digit_Recognizer/Data/test.csv"
test_df = pd.read_csv(path_test)
test_df.info()

In [ ]:
test_df.head()

In [ ]:
test_data = test_df.values.reshape(-1, 28, 28, 1)
test_data = X_test/255.0
test_data.shape

In [ ]:
plt.figure(figsize=(6,6))
for num in range(0,25):
    plt.subplot(5,5,num+1)
    plt.imshow(test_data[num, :, :, :], interpolation = "none", cmap = plt.cm.binary)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Define model
def build_model():
    model = tf_keras.models.Sequential([
    tf_keras.layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=[28, 28, 1]),
    tf_keras.layers.MaxPooling2D(2),
    tf_keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    tf_keras.layers.MaxPooling2D(2),
    tf_keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf_keras.layers.MaxPooling2D(2),
    tf_keras.layers.Flatten(),
    tf_keras.layers.Dense(512, activation='relu'),
    tf_keras.layers.Dropout(rate=0.3),
    tf_keras.layers.Dense(256, activation='relu'),
    tf_keras.layers.Dropout(rate=0.3),
    tf_keras.layers.Dense(128, activation='relu'),
    tf_keras.layers.Dropout(rate=0.3),
    tf_keras.layers.Dense(10, activation='softmax')
])


    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    return model

In [ ]:
from tensorflow.keras import callbacks

# Define early stopping callback on validation loss
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
)

# Reduce learning rate when validation loss plateaus
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5
)

In [ ]:
#Cross validation
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score

FOLDS = 10
EPOCHS = 200
BATCH_SIZE = 500

test_preds = np.zeros((1, 1))
scores = []
times = []
history_df = pd.DataFrame()

cv = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=0)

for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    # Start timer
    start = time.time()

    # get training and validation sets
    X_train, X_valid = X[train_idx], X[val_idx]
    y_train, y_valid = y[train_idx], y[val_idx]

    # Build and train model
    model = build_model()
    fold_history = model.fit(
        X_train,
        y_train,
        validation_data=(X_valid, y_valid),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        #callbacks=[early_stopping, reduce_lr],
        verbose=False
    )

    history_df = pd.concat([history_df, pd.DataFrame(fold_history.history)], ignore_index=True)


    # Make predictions and measure accuracy
    y_pred = np.argmax(model.predict(X_valid), axis=1)
    score = accuracy_score(y_valid, y_pred)
    scores.append(score)

    # Store predictions
    test_preds = test_preds + model.predict(test_data)

    # Stop timer
    stop = time.time()
    times.append((stop - start)/60)

    # Print accuracy and time
    print(f'Fold {fold} - Accuracy: {score}, Time: {round((stop - start)/60,1)} mins')

print('')
print(f'Mean Accuracy: {np.mean(scores)}')

In [ ]:
#Plot fold accuracy

# Figure size
plt.figure(figsize=(10,5))

# Line plot
plt.plot(scores, label='scores')
plt.plot(np.mean(scores)*np.ones(len(scores)), c='black', label='mean')

# Aesthetics
plt.title('Fold accuracy')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.ylim([0.99,1])
plt.legend()

In [ ]:
#Plot fold time

# Figure size
plt.figure(figsize=(10,5))

# Line plot
plt.plot(times, label='times', c='orange')
plt.plot(np.mean(times)*np.ones(len(times)), c='black', label='mean')

# Aesthetics
plt.title('Fold time')
plt.xlabel('Fold')
plt.ylabel('Time (mins)')
plt.legend()

In [ ]:
#Plot learning curves
for i in range(FOLDS):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,4))
    plt.suptitle('Fold '+str(i+1), fontsize=20)

    plt.subplot(1,2,1)
    ax=history_df.reset_index().loc[EPOCHS*i+1:EPOCHS*(i+1)-1, ['loss', 'val_loss']].plot(title="Cross-entropy", ax=axes[0])
    plt.xlabel('Epoch')

    plt.subplot(1,2,2)
    ax=history_df.reset_index().loc[EPOCHS*i+1:EPOCHS*(i+1)-1, ['accuracy', 'val_accuracy']].plot(title="Accuracy", ax=axes[1])
    plt.xlabel('Epoch')


In [ ]:
# Soft voting to ensemble predictions
test_preds = np.argmax(test_preds, axis=1)

In [ ]:
# Plot some model predictions
plt.figure(figsize=(15,3.5))
plt.suptitle('Model predictions', fontsize=20, y=1.05)

# Subplot
for i in range(20):
    img = test_data[i];
    ax=plt.subplot(2, 10, i+1)
    ax.grid(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.title.set_text(f'Pred:{test_preds[i]}')
    plt.imshow(img, cmap='gray')
    
plt.show()

In [ ]:
# Save predictions to file
sample_sub = pd.read_csv("/content/sample_data/Digit_Recognizer/sample_submission.csv")
test_id = sample_sub.ImageId.copy()

output = pd.DataFrame({'ImageId': test_id,
                       'Label': test_preds})

# Check format
output.head()

In [ ]:
output.to_csv('/content/sample_data/Digit_Recognizer/submission-03.csv', index=False)